In [1]:
from io import StringIO

import anndata
import pandas as pd
import numpy as np
import scanpy
import scanpy as sc

scanpy.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
scanpy.logging.print_versions()
scanpy.set_figure_params(frameon=False, color_map='magma_r')

outdir = '/home/olga/googledrive/TabulaMicrocebus/data/cross-species'
! ls -lha $outdir

scanpy==1.4.6.dev9+ge632939 anndata==0.6.22.post1 umap==0.3.10 numpy==1.17.3 scipy==1.3.2 pandas==0.25.3 scikit-learn==0.22 statsmodels==0.10.2 python-igraph==0.7.1 louvain==0.6.1
total 41G
drwxrwxr-x 1 olga czb  4.0K Feb 13 11:27  .
drwxrwxr-x 1 olga czb  4.0K Dec  2 21:43  ..
drwxrwxr-x 1 olga czb  4.0K Mar 21 21:55  alltissues__10x__lemur__one2one_orthologs__csvs
-rw-r--r-- 1 olga czb  1.8G May 19 08:56  alltissues__10x__lemur__one2one_orthologs.h5ad
drwxrwxr-x 1 olga czb  4.0K Mar 21 16:51  alltissues__10x__mouse__one2one_orthologs__csvs
-rw-r--r-- 1 olga czb  2.5G May 19 08:54  alltissues__10x__mouse__one2one_orthologs.h5ad
drwxr-xr-x 1 olga czb  4.0K May 20 12:36  binarized
-r--r--r-- 1 olga czb   195 May 19 16:37 'binarized_expression__stable_genes__compartment_conserved (564988b6).desktop'
-rw-r--r-- 1 olga czb   77K May 19 08:00  binarized_expression__stable_genes__compartment_conserved.csv
-r--r--r-- 1 olga czb   195 May 19 08:18  binarized_expression__stable_genes__compartme

# Define outdirs

In [2]:
outdir_gdrive = '/home/olga/googledrive/TabulaMicrocebus/data/cross-species/within-species-de'
outdir_local = '/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/within-species-de'
! mkdir $outdir_gdrive $outdir_local
outdirs = outdir_local, outdir_gdrive
!ls -lha $outdir_local

mkdir: cannot create directory ‘/home/olga/googledrive/TabulaMicrocebus/data/cross-species/within-species-de’: File exists
mkdir: cannot create directory ‘/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/within-species-de’: File exists
total 2.8G
drwxr-xr-x  2 olga czb 4.0K May 24 09:02 .
drwxrwxr-x 13 olga czb 8.0K May 22 13:37 ..
-rw-r--r--  1 olga czb 553M May 24 08:53 lemur_lung__diffexpr_compartment_group__defaults.h5ad
-rw-r--r--  1 olga czb 553M May 22 15:58 lemur_lung__diffexpr_compartment_group.h5ad
-rw-r--r--  1 olga czb 426M May 24 09:02 lemur_lung__diffexpr_narrow_group__defaults.h5ad
-rw-r--r--  1 olga czb 307M May 24 08:53 mouse_lung__diffexpr_broad_group__defaults.h5ad
-rw-r--r--  1 olga czb 307M May 23 00:02 mouse_lung__diffexpr_broad_group.h5ad
-rw-r--r--  1 olga czb 307M May 24 08:43 mouse_lung__diffexpr_compartment_group__defaults.h5ad
-rw-r--r--  1 olga czb 307M May 24 08:49 mouse_lung__diffexpr_narrow_group__defaults.h5ad
-rw-r--r--  1 o

# Load into Scanpy

## Read preprocessed Human h5ad

In [3]:
human_folder = '/home/olga/googledrive/TabulaMicrocebus/data/human-lung-cell-atlas--from-kyle'
! ls -lha $human_folder

total 24G
drwxrwxr-x 1 olga czb 4.0K Dec 13 11:44 .
drwxrwxr-x 1 olga czb 4.0K Dec  2 21:43 ..
-rw-rw-r-- 1 olga czb 6.5G Dec 14 19:07 droplet_normal_lung_blood_P1-3.h5ad
-rw-rw-r-- 1 olga czb  12M Oct 28  2019 droplet_normal_lung_blood_P1-3_metadata.csv
-rw-rw-r-- 1 olga czb 3.3G Oct 30  2019 droplet_normal_lung_blood_P1-3_ncbi_GCT.csv
-rw-r--r-- 1 olga czb 6.2G May 18 15:15 droplet_normal_lung_blood_P1-3__no_duplicate_barcodes.h5ad
-rw-r--r-- 1 olga czb 6.5G May 18 15:11 droplet_normal_lung_blood_P1-3__with_barcode_counts.h5ad
-rw-rw-r-- 1 olga czb 1.1G Oct 28  2019 facs_normal_lung_blood_P1-3_gencode_GCT.csv
-rw-rw-r-- 1 olga czb 5.7M Oct 28  2019 facs_normal_lung_blood_P1-3_metadata.csv


In [4]:
%%time

h5ad = f"{human_folder}/droplet_normal_lung_blood_P1-3.h5ad"
human = scanpy.read(h5ad, cache=True)
human

CPU times: user 678 ms, sys: 6.46 s, total: 7.14 s
Wall time: 1min 28s


AnnData object with n_obs × n_vars = 65662 × 26485 
    obs: 'nGene', 'nUMI', 'orig.ident', 'channel', 'tissue', 'region', 'percent.ribo', 'free_annotation', 'patient', 'sample', 'location', 'magnetic.selection', 'preparation.site', 'compartment', 'species', 'species_latin', 'cell_ontology_class'

## Subset to lung

In [5]:
human_lung = human[human.obs.tissue == "lung"]
human_lung

View of AnnData object with n_obs × n_vars = 60993 × 26485 
    obs: 'nGene', 'nUMI', 'orig.ident', 'channel', 'tissue', 'region', 'percent.ribo', 'free_annotation', 'patient', 'sample', 'location', 'magnetic.selection', 'preparation.site', 'compartment', 'species', 'species_latin', 'cell_ontology_class'

### Look at lemur lung.obs.head()

In [6]:
human_lung.obs.head()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,species,species_latin,cell_ontology_class
index,,,,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell


## Read all species object with unified compartments

In [16]:
%%time
h5ad = f'{outdir}/concatenated__human-lung--lemur-lung--mouse-lung__10x__one2one_orthologs__unified_compartments.h5ad'
adata = scanpy.read_h5ad(h5ad)
adata

CPU times: user 773 ms, sys: 967 ms, total: 1.74 s
Wall time: 5.42 s


AnnData object with n_obs × n_vars = 118089 × 11970 
    obs: 'age', 'batch', 'cell', 'cell_barcode', 'cell_ontology_class', 'cell_ontology_id', 'channel', 'channel_no_10x', 'channel_tissue', 'compartment', 'compartment_id', 'free_annotation', 'individual', 'location', 'magnetic.selection', 'method', 'mouse.id', 'nGene', 'nUMI', 'n_barcodes', 'n_counts', 'n_genes', 'orig.ident', 'original_channel', 'patient', 'percent.ribo', 'possibly_contaminated_barcode', 'preparation.site', 'region', 'sample', 'sequencing_run', 'sex', 'species', 'species_batch', 'species_latin', 'subtissue', 'tissue', 'tissue_free_annotation', 'compartment_updated_from_human', 'compartment_updated_from_human__with_species', 'compartment_updated_from_lemur', 'compartment_updated_from_lemur__with_species', 'narrow_group', 'broad_group', 'compartment_group'
    var: 'mouse_lemur__gene_name', 'mouse_lemur_to_human__homology_type', 'human_to_mouse__homology_type', 'mouse__gene_name', 'gene_ids-lemur', 'n_cells-mouse'

### Look at adata.obs.head()

In [17]:
adata.obs.head()

,age,batch,cell,cell_barcode,cell_ontology_class,cell_ontology_id,channel,channel_no_10x,channel_tissue,compartment,...,subtissue,tissue,tissue_free_annotation,compartment_updated_from_human,compartment_updated_from_human__with_species,compartment_updated_from_lemur,compartment_updated_from_lemur__with_species,narrow_group,broad_group,compartment_group
P2_1_AAACCTGAGAAACCAT-human,NaN,NaN,NaN,AAACCTGAGAAACCAT,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AAATGCCAGATGAGAG-human,NaN,NaN,NaN,AAATGCCAGATGAGAG,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AACACGTTCGATCCCT-human,NaN,NaN,NaN,AACACGTTCGATCCCT,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AACACGTTCGCACTCT-human,NaN,NaN,NaN,AACACGTTCGCACTCT,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AACCATGCAGCTCGCA-human,NaN,NaN,NaN,AACCATGCAGCTCGCA,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial


### Remove "-human" from combined annotations

In [18]:
obs_from_concatenated = adata.obs.query('species_batch == "human"')
obs_from_concatenated.index = obs_from_concatenated.index.str.split('-human').str[0]
obs_from_concatenated.head()

,age,batch,cell,cell_barcode,cell_ontology_class,cell_ontology_id,channel,channel_no_10x,channel_tissue,compartment,...,subtissue,tissue,tissue_free_annotation,compartment_updated_from_human,compartment_updated_from_human__with_species,compartment_updated_from_lemur,compartment_updated_from_lemur__with_species,narrow_group,broad_group,compartment_group
P2_1_AAACCTGAGAAACCAT,NaN,NaN,NaN,AAACCTGAGAAACCAT,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AAATGCCAGATGAGAG,NaN,NaN,NaN,AAATGCCAGATGAGAG,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AACACGTTCGATCCCT,NaN,NaN,NaN,AACACGTTCGATCCCT,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AACACGTTCGCACTCT,NaN,NaN,NaN,AACACGTTCGCACTCT,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial
P2_1_AACCATGCAGCTCGCA,NaN,NaN,NaN,AACCATGCAGCTCGCA,capillary endothelial cell,NaN,P2_1,NaN,NaN,endothelial,...,NaN,Lung,NaN,endothelial,endothelial (Human),endothelial,endothelial (Human),Capillary Aerocyte,Capillary,endothelial


## Add unified compartments

In [19]:
unified_cols = ['narrow_group', 'broad_group', 'compartment_group']

In [20]:
obs_with_compartments = human_lung.obs.join(obs_from_concatenated[unified_cols].astype(str))
obs_with_compartments.head()

,nGene,nUMI,orig.ident,channel,tissue,region,percent.ribo,free_annotation,patient,sample,location,magnetic.selection,preparation.site,compartment,species,species_latin,cell_ontology_class,narrow_group,broad_group,compartment_group
index,,,,,,,,,,,,,,,,,,,,
P2_1_AAACCTGAGAAACCAT,1347,2914,Human Lung 10x - Patient 2,P2_1,lung,normal,0.035347,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell,Capillary Aerocyte,Capillary,endothelial
P2_1_AAATGCCAGATGAGAG,1713,4226,Human Lung 10x - Patient 2,P2_1,lung,normal,0.061051,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell,Capillary Aerocyte,Capillary,endothelial
P2_1_AACACGTTCGATCCCT,1185,2152,Human Lung 10x - Patient 2,P2_1,lung,normal,0.048327,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell,Capillary Aerocyte,Capillary,endothelial
P2_1_AACACGTTCGCACTCT,1378,3419,Human Lung 10x - Patient 2,P2_1,lung,normal,0.032758,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell,Capillary Aerocyte,Capillary,endothelial
P2_1_AACCATGCAGCTCGCA,1210,2514,Human Lung 10x - Patient 2,P2_1,lung,normal,0.050119,Capillary Aerocyte,2,distal 2,distal,epithelial,biohub,endothelial,Human,Homo sapiens,capillary endothelial cell,Capillary Aerocyte,Capillary,endothelial


In [21]:
obs_with_compartments[unified_cols].notnull().sum()

narrow_group         57353
broad_group          57353
compartment_group    57353
dtype: int64

In [22]:
obs_with_compartments['compartment_group'].value_counts()

myeloid                    18854
endothelial                15626
lymphoid                    9406
epithelial                  8737
stromal                     4710
megakaryocyte-erythroid       10
neural                        10
Name: compartment_group, dtype: int64

In [23]:
obs_with_compartments['narrow_group'].value_counts()

Macrophage                                13951
Capillary                                  6952
Alveolar Epithelial Type 2                 4225
Capillary Aerocyte                         4061
Natural Killer                             3974
CD4+ T                                     2543
CD8+ T                                     2068
Monocyte                                   1893
Basophil                                   1798
Pericyte                                   1490
Artery                                     1398
Ciliated                                   1312
Alveolar Fibroblast                        1210
Vein                                       1100
Capillary Intermediate                     1022
Alveolar Epithelial Type 1                  904
Airway Smooth Muscle and Myofibroblast      886
Club                                        881
Basal                                       770
Bronchial Vessel                            654
Adventitial Fibroblast                  

In [25]:
human_lung_new_obs = human_lung.copy()
human_lung_new_obs.obs = obs_with_compartments
human_lung_new_obs = human_lung_new_obs[human_lung_new_obs.obs.groupby('narrow_group').filter(lambda x: len(x) >= 3).index]
human_lung_new_obs = human_lung_new_obs[human_lung_new_obs.obs.query('narrow_group != "nan"').index]


## Write newly created object to file

In [26]:
for d in outdirs:
    h5ad = f'{d}/human_lung_from_hlca__unified_compartments.h5ad'
    print(h5ad)
    human_lung_new_obs.write(h5ad)

/home/olga/data_lg/data_sm_copy/tabula-microcebus/data-objects/cross-species/within-species-de/human_lung_from_hlca__unified_compartments.h5ad


/home/olga/miniconda3/envs/tabula-microcebus-v2/lib/python3.7/site-packages/anndata/_core/anndata.py:1068: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning,
Trying to set attribute `.obs` of view, copying.
... storing 'narrow_group' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'broad_group' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'compartment_group' as categorical


/home/olga/googledrive/TabulaMicrocebus/data/cross-species/within-species-de/human_lung_from_hlca__unified_compartments.h5ad


# do differential expression


In [27]:
import diffexpr

In [28]:
# human_lung_new_obs.uns['rank_genes_groups']

In [29]:
prefix = 'human_lung'

## narrow group

In [30]:
%%time

group = 'narrow_group'
sc.tl.rank_genes_groups(human_lung_new_obs, group)

for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}__defaults.h5ad"
    human_lung_new_obs.write(h5ad)

# mouse_lung_new_obs.uns['rank_genes_groups']

# sc.pl.rank_genes_groups(mouse_lung_new_obs, sharey=False)


ranking genes


/home/olga/miniconda3/envs/tabula-microcebus-v2/lib/python3.7/site-packages/scanpy/tools/_rank_genes_groups.py:237: RuntimeWarning: overflow encountered in expm1
  foldchanges = (expm1_func(mean_group) + 1e-9) / (expm1_func(mean_rest) + 1e-9)  # add small value to remove 0's


    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:03:16)
CPU times: user 2min 15s, sys: 1min 8s, total: 3min 23s
Wall time: 4min 54s


## broad group

In [31]:
%%time

group = 'broad_group'
sc.tl.rank_genes_groups(human_lung_new_obs, group)

for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}__defaults.h5ad"
    human_lung_new_obs.write(h5ad)

# mouse_lung_new_obs.uns['rank_genes_groups']

# sc.pl.rank_genes_groups(mouse_lung_new_obs, sharey=False)


ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:02:16)
CPU times: user 1min 32s, sys: 51.7 s, total: 2min 24s
Wall time: 5min 38s


## compartment group

In [32]:
%%time

group = 'compartment_group'
sc.tl.rank_genes_groups(human_lung_new_obs, group)

for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}__defaults.h5ad"
    human_lung_new_obs.write(h5ad)

# mouse_lung_new_obs.uns['rank_genes_groups']

# sc.pl.rank_genes_groups(mouse_lung_new_obs, sharey=False)


ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:09:26)
CPU times: user 30.4 s, sys: 9min 1s, total: 9min 32s
Wall time: 11min 12s


# Logistic regression

## compartment group

In [ ]:
%%time

group = 'compartment_group'
sc.tl.rank_genes_groups(human_lung_new_obs, group)

sc.pl.rank_genes_groups(human_lung_new_obs, sharey=False)


In [ ]:
group = 'compartment_group'

diffexpr.do_differential_expression(human_lung_new_obs, group, n_jobs=16)

ranking genes


### Write results to file

In [ ]:
for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}.h5ad"
    human_lung_new_obs.write(h5ad)
#     human_lung_new_obs.write_csvs(f'{prefix}__diffexpr__{group}')

## narrow group

In [ ]:
group = 'narrow_group'
diffexpr.do_differential_expression(human_lung_new_obs, group) 

### Write results to file

In [ ]:
for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}.h5ad"
    human_lung_new_obs.write(h5ad)
#     human_lung_new_obs.write_csvs(f'{prefix}__diffexpr__{group}')

## Broad group

In [ ]:
group = 'broad_group'

do_differential_expression(human_lung_new_obs, group)

In [ ]:
human_lung_new_obs.uns['rank_genes_groups']

### Write results to file

In [ ]:
for d in outdirs:
    h5ad = f"{d}/{prefix}__diffexpr_{group}.h5ad"
    human_lung_new_obs.write(h5ad)
#     human_lung_new_obs.write_csvs(f'{prefix}__diffexpr__{group}')